---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import sklearn as sk
import math
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score 

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
G = nx.read_gpickle('email_prediction.txt')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [3]:
def salary_predictions():
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    df['Clustering'] = pd.Series(nx.clustering(G))
    df['Degree'] = pd.Series(G.degree())
    df['Betweenness'] = nx.betweenness_centrality(G, normalized=True, endpoints=False).values()
    df['Closeness'] = nx.closeness_centrality(G).values()
    
    train_set = df[df['ManagementSalary'].isin([0.0, 1.0])]
    test_set = df[np.isnan(df['ManagementSalary'])]
    train_X = train_set[['Clustering', 'Degree', 'Betweenness', 'Closeness']]
    train_y = train_set['ManagementSalary']
    test_X = test_set[['Clustering', 'Degree', 'Betweenness', 'Closeness']]
    
    LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(train_X, train_y)  
    probability = LR.predict_proba(test_X)[:,1] #[:,1] - probability of receiving [:,0] - probability of not receiving
    test_set['ManagementSalary'] = np.round(np.array(probability),1)

    return test_set['ManagementSalary']

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [4]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [5]:
def new_connections_predictions():
    df_jc = pd.DataFrame()
    df_aa = pd.DataFrame()

    jaccard_coeff = list(nx.jaccard_coefficient(G))
    df_jc['edge'] = [(tup[0], tup[1]) for tup in jaccard_coeff]
    df_jc['jaccard_coeff'] = [tup[2] for tup in jaccard_coeff]
    df_jc = df_jc.set_index('edge', drop=True)
    del df_jc.index.name

    adamic_adar = list(nx.adamic_adar_index(G))
    df_aa['edge'] = [(tup[0], tup[1]) for tup in adamic_adar]
    df_aa['adamic_adar'] = [tup[2] for tup in adamic_adar]
    df_aa = df_aa.set_index('edge', drop=True)
    del df_aa.index.name
    
    df = df_aa.merge(df_jc, left_index=True, right_index=True)
    
    train_set = future_connections[future_connections['Future Connection'].isin([0.0, 1.0])]
    train_set = train_set.merge(df, left_index=True, right_index=True)
    
    test_set = future_connections[np.isnan(future_connections['Future Connection'])]
    test_set = test_set.merge(df, left_index=True, right_index=True)
    
    features = ['jaccard_coeff', 'adamic_adar']
    train_X = train_set[features]
    train_y = train_set[['Future Connection']]
    test_X = test_set[features]
    
    scaler = MinMaxScaler()
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.transform(test_X)
    
    LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(train_X, train_y)  
    probability = LR.predict_proba(test_X)[:,1] #[:,1] - probability of connecting [:,0] - probability of not connecting
    test_set['Future Connection'] = np.round(np.array(probability),2)
    
    return test_set['Future Connection']